<a href="https://colab.research.google.com/github/A47126/BT-Machine-Learning-Group-7/blob/main/mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mô tả & Tiền xử lý

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('mushrooms.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'mushrooms.csv'

In [ ]:
df.shape
# 8124 hàng, 23 cột

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()
# Không có dữ liệu null

In [ ]:
df.duplicated()
# Không có dữ liệu trùng

In [ ]:
df.nunique()
# Cột veil-type có duy nhất 1 giá trị => có thể bỏ mà không ảnh hưởng kết quả mô hình

In [ ]:
le = LabelEncoder()
for col in df.columns:
    df[col] = le.fit_transform(df[col])

In [ ]:
for cols in df.columns:
    plt.figure(figsize=(8,4))
    df[cols].value_counts().plot(kind='bar')
    plt.title(f'Tần suất xuất hiện - Cột {cols}')
    plt.xlabel('Giá trị')
    plt.ylabel('Số lần xuất hiện')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
df = df.drop(columns = ['veil-type'])

In [ ]:
df['class'].value_counts()
# Cột nhãn khá cân bằng

# Chia train, validation, test

In [ ]:
X = df.drop(columns=['class'])
y = df['class']

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)
X_train.shape, X_val.shape, X_test.shape

In [ ]:
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_train_resamp, y_train_resamp = smote.fit_resample(X_train, y_train)

# A47126: Decision Tree


1. Import thư viện mô hình & các thư viện cần thiết khác

2. Huấn luyện mô hình

3. Dự đoán

4. Tối ưu hóa mô hình

5. Đánh giá mô hình (LƯU Ý: giảm False Negative) & phải có Confusion Matrix, AUC

# A47047: Logistics Regression

1. Import thư viện mô hình & các thư viện cần thiết khác

In [ ]:
from sklearn.linear_model import LogisticRegression

2. Huấn luyện mô hình

3. Dự đoán

4. Tối ưu hóa mô hình

5. Đánh giá mô hình (LƯU Ý: Dùng Recall để giảm False Negative) & phải có Confusion Matrix, AUC

# A49569: K-nearest neighbors

1. Import thư viện mô hình & các thư viện cần thiết khác

2. Huấn luyện mô hình

3. Dự đoán

4. Tối ưu hóa mô hình

5. Đánh giá mô hình (LƯU Ý: Dùng Recall để giảm False Negative) & phải có Confusion Matrix, AUC

# A49002: Bernoulli, Categorical

1. Import thư viện mô hình & các thư viện cần thiết khác

2. Huấn luyện mô hình

3. Dự đoán

4. Tối ưu hóa mô hình

5. Đánh giá mô hình (LƯU Ý: Dùng Recall để giảm False Negative) & phải có Confusion Matrix, AUC

# A48572: Gaussian, Multinomial

1. Import thư viện mô hình & các thư viện cần thiết khác

2. Huấn luyện mô hình

3. Dự đoán

4. Tối ưu hóa mô hình

5. Đánh giá mô hình (LƯU Ý: Dùng Recall để giảm False Negative) & phải có Confusion Matrix, AUC

# Lưu code

In [ ]:
# Viết xong: Ctrl + S
# Sau đó
# Mở terminal lên: Ctrl + ~
# Copy câu này vào: git add . && git commit -m "Up" && git push